In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import TiDBVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

import getpass
import os

tidb_connection_string = getpass.getpass("TiDB Connection String:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Creates the vectors stored in TidB from the text file
# docs = None

# with open ("country_data_markdown.txt", "r", encoding="UTF-8") as file:
#     document = file.read()

#     headers_to_split_on = [
#         ('#', "Country"),
#     ]

#     markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
#     docs = markdown_splitter.split_text(document)

# with open ("country_metadata.txt", 'r', encoding="UTF-8") as file:
#     docsIndex = 0
#     for line in file:
#         line = line.strip()
#         if not line:
#             docsIndex += 1
#             continue
#         if ":" in line:
#             key, value = line.split(":", 1)
#             key = key.strip()
#             value = value.strip()

#             if key == "Crime_Index" or key == "Download_Speed" or key == "Tap_Water_Index":
#                 docs[docsIndex].metadata[key] = float(value)
#             else:
#                 docs[docsIndex].metadata[key] = value

embeddings = OpenAIEmbeddings()

# Creates the Vector Store and saves it to TiDB
# vector_store = TiDBVectorStore.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     table_name="vectors",
#     connection_string=tidb_connection_string,
#     distance_strategy="cosine", 
# )

# Query the Vector Store
vector_store = TiDBVectorStore.from_existing_vector_table(
    embedding=embeddings,
    connection_string=tidb_connection_string,
    table_name="vectors",
    distance_strategy="cosine",
)

# # Finds the most similar document to the query
query = "Find a country with people that speak English and Spanish, with warm weather, with extra hot spicy food, with people that follow Christianity, with a crime index of under 6, with landmarks, with many places for hiking, with broadband download speed of over 50 Mbps, with a tap water index of over 60, with no ongoing conflicts or regional tensions, with political stability and no political tensions, with a government that has a voting system, not in the continents of North America, and specifically not Bangladesh, Libya, Lebanon, Afghanistan, Somalia, Iran, Yemen, Syria, Russia, Myanmar, Venezuela, Iraq, South Sudan, Mali, Central African Republic, Burkina Faso, Haiti, Belarus, North Korea, Ukraine, Sudan, Mexico, Israel, or Palestine State."
filters = {
    "Crime_Index":{"$lt": 4.5},
    "Download_Speed":{"$gt": 100},
    "Tap_Water_Index":{"$gt": 80},
}

docs_with_score = vector_store.similarity_search_with_relevance_scores(query, filter=filters, k=20)
# docs_with_score = vector_store.similarity_search_with_relevance_scores(query, k=20)
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    # print(doc.metadata)
    print("-" * 80)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)